# n-gram Language Modeling

In this part of the assignment, we'll expand on the `SimpleTrigramLM` from the live session demo. We'll add smoothing to improve performance on unseen data, and explore some of the properties of the smoothed model.

If you haven't looked over the simple trigram LM in [lm1.ipynb](../../materials/simple_lm/lm1.ipynb), we recommend familiarizing yourself with it. This assignment will use a very similar set-up.

## A note on notation

"Primed" variables, e.g. $c′$. This is just a dummy variable, not necessarily equal to $c$ but usually assumed to be from the same domain. For example: $P(c)$ is the probability of a particular word $c$, while $\sum_{c′} P(c′)$ is the sum of the probabilities of all possible words $c′$. If not explicitly stated, the domain is usually the domain of the expression inside the sum - so in this case we'd sum for all words $c′ \in V$ in the vocabulary.
 
- Set notation: $\{x : f(x) > 0\}$ means the set of all $x$ where $f(x)>0$. Strictly speaking, we should write $\{x∈S:f(x)>0\}$ where $S$ is some other set, but often we omit this when $S$ is implied (such as words (types) in the vocabulary). So $\{ b′:C_{b′c} > 0 \}$ is the set of all words (types) $b′$ where the counts $C_{b′c}$ (for some particular word $c$) are greater than zero.
 
- Similar to the above, $\left| x:f(x)>0 \right|$ means the number of elements (size) of that set. So $\left| b′:C_{b′c} > 0 \right|$ is the number of words $b′$ where the counts $C_{b′c}$ (for some particular word $c$) are greater than zero.

In [36]:
import os, sys, re, json, time, unittest
import itertools, collections
from importlib import reload

import numpy as np
from scipy import stats

import nltk

# Helper libraries for this notebook
from w266_common import utils, vocabulary
import ngram_lm, ngram_lm_test

# Add-k Smoothing

Recall our unsmoothed maximum likelihood estimate of $ P(w_i\ |\ w_{i-1}, w_{i-2})$ where we use the raw distribution over words seen in a context in the training data:

$$  \hat{P}(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc}}{\sum_{c'} C_{abc'}} $$

Add-k smoothing is the simple refinement where we add $k > 0$ to each count $C_{abc}$, pretending we've seen every vocabulary word $k$ extra times in each context. So we have:

$$ \hat{P}_k(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{C_{abc} + k}{\sum_{c'} (C_{abc'} + k)} = \frac{C_{abc} + k}{C_{ab} + k\cdot|V|} $$

where $|V|$ is the size of our vocabulary.

In the questions below and in the code, we'll refer to $(w_{i-2}, w_{i-1})$ as the *context*, and $w_i$ as the current *word*. By convention, we'll somewhat interchangeably refer to the sequence $(w_{i-2}, w_{i-1}, w)$ as $abc$.

### Part (a): Short answer questions

Give brief answers to the following, in the cell below.

1. If we encounter a new context `(foo, bar)` unseen in the training data, what will the predicted *distribution* $\hat{P}_k(w\ |\ \text{foo}, \text{bar})$ be? How does your answer depend on $k$?
<p>
2. If we encounter a new word in a familiar context (i.e. `ab` is in the corpus, but `abq` is not), what will our predicted probability $\hat{P}_k(q\ |\ b, a)$ be? Assume $C_{abq} = 0$, $C_{ab}=10$, $k=2$, and $|V|=2000$.
<p>
3. Based on your answer to question 2, in which context will your model predict a higher probability of *any* unknown word?  
Context (a): `<s> the ___`  
Context (b): `Mister Rogers ___`  
Assume $C_{\text{<s>, the}} = 10000$ and $C_{\text{Mister, Rogers}} = 47$.
<p>
4. Based on your knowledge of language, which of the contexts from question 3 *should* have a higher probability of an unknown word?  _(Hint: recall that most unknown tokens are nouns)_

 answer:
 1. k/(k|V|)
 2. (C_abq+k)/(C_ab+k|V|) = (0+2)/(10+2*2000) = 4.9875e-4

In [2]:
(0+2)/(10+2*2000)

0.0004987531172069825

In [3]:
(0+2)/(10000+2*2000)

0.00014285714285714287

In [4]:
(0+2)/(47+2*2000)

0.0004941932295527551

## Part (b): Implementing the Add-k Model

Despite its shortcomings, it's worth implementing an add-k model as a baseline. Unlike the unsmoothed model, we'll be able to get some reasonable (or at least, finite) perplexity numbers which we can compare to the Kneser-Ney model below.

We've provided some skeleton code (similar to [lm1.ipynb](../../materials/simple_lm/lm1.ipynb)) in the `ngram_lm.py` file. In the `AddKTrigramLM` class, implement the following:
- `__init__(self, words)`, which computes the necessary corpus statistics $C_{abc}$ and $C_{ab}$.
- `next_word_proba(self, word, seq, k)`, which computes $\hat{P}_k(w\ |\ w_{i-1}, w_{i-2})$

See the function docstrings and in-line comments for more details. In particular, you may want to use `collections.defaultdict` and `dict.get()` to simplify handling of unknown keys. See [dict_notes.md](dict_notes.md) for a brief overview of how to use these.

**Note on keys and word-order:** Convention in the math is to write context in reverse order, as in $P(w\ |\ w_{i-1}, w_{i-2})$, but in the code it'll be much easier to write things left-to-right as in normal English: so for the context "`foo bar ___`", you'll want to use `("foo", "bar")` as a dict key.



In [37]:
reload(ngram_lm)
utils.run_tests(ngram_lm_test, ["TestAddKTrigramLM"])

test_context_totals (ngram_lm_test.TestAddKTrigramLM) ... ok
test_counts (ngram_lm_test.TestAddKTrigramLM) ... ok
test_next_word_proba_k_exists (ngram_lm_test.TestAddKTrigramLM) ... ok
test_next_word_proba_no_smoothing (ngram_lm_test.TestAddKTrigramLM) ... ok
test_no_mutate_on_predict (ngram_lm_test.TestAddKTrigramLM) ... ok
test_words (ngram_lm_test.TestAddKTrigramLM) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


# Training your Model

The same code below can be used with either model; in the cell where it says "Select your Model", you can choose the add-k model or the KN model.

## Loading & Preprocessing
Once again, we'll build our model on the Brown corpus. We'll do an 80/20 train/test split, and preprocess words by lowercasing and replacing digits with `DG` (so `2016` becomes `DGDGDGDG`).

In a slight departure from the `lm1.ipynb` demo, we'll restrict the vocabulary to 30000 words. This way, a small fraction of the *training* data will be mapped to `<unk>` tokens, and the model can learn n-gram probabilities that include `<unk>` for prediction on the test set. (If we interpret `<unk>` as meaning "rare word", then this is somewhat plausible as a way to infer things about the class of rare words.)

In [38]:
assert(nltk.download('brown'))  # Make sure we have the data.
corpus = nltk.corpus.brown
V = 30000
train_sents, test_sents = utils.get_train_test_sents(corpus, split=0.8, shuffle=False)
# Build vocabulary only on the training set.
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(train_sents)), size=V)
print("Train set vocabulary: {:,} words".format(vocab.size))

[nltk_data] Downloading package brown to /home/haoyuzhang/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Loaded 57,340 sentences (1.16119e+06 tokens)
Training set: 45,872 sentences (979,646 tokens)
Test set: 11,468 sentences (181,546 tokens)
Train set vocabulary: 30,000 words


Our smoothed models will also be trigram models, so for convenience we'll also prepend *two* `<s>` markers. (We could avoid this, but then we'd need special handling for the first token of each sentence.)

To make it easier to work with, we'll take the list of tokens as a NumPy array.

In [39]:
def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = ([u"<s>", u"<s>"] + s + [u"</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in utils.flatten(padded_sentences)], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)
print("Sample data: \n", repr(train_tokens[:20]))

Sample data: 
 array(['<s>', '<s>', 'the', 'fulton', 'county', 'grand', 'jury', 'said',
       'friday', 'an', 'investigation', 'of', "atlanta's", 'recent',
       'primary', 'election', 'produced', '``', 'no', 'evidence'],
      dtype=object)


## Select your model

In [40]:
reload(ngram_lm)

Model = ngram_lm.AddKTrigramLM

t0 = time.time()
print("Building trigram LM... ", end="")
lm = Model(train_tokens)
print("done in {:.02f} s".format(time.time() - t0))
lm.print_stats()

Building trigram LM... done in 4.05 s
=== N-gram Language Model stats ===
       0 unique 1-grams
       0 unique 2-grams
 732,467 unique 3-grams
Optimal memory usage (counts only): 16.70 MB


In [41]:
lm.set_live_params(k = 0.001)

## Sampling Sentences

In [42]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(lm.sample_next(seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print(" ".join(seq))
    print("[{1:d} tokens; log P(seq): {0:.02f}]\n".format(*lm.score_seq(seq)))

<s> <s> some would suicide hockaday voroshilov endowment scorn scalar locomotive yardumian dogleg unmarried london anti-aircraft backdrop increasingly barrel statue efficiency drury
[20 tokens; log P(seq): -283.50]

<s> <s> four nick's gratified constancy swim framed bronzy-green-gold richter-haaser confounded fund-raising flat-bottomed hydrogen guise willed collections pollock culminate q3 detective's cerv
[20 tokens; log P(seq): -294.44]

<s> <s> you couldn't illustrated no-nonsense performance luechtefeld uk romantics basic magazines stains qualifications bears switching undermining flanked tasmania deliberately alcoholics anticipated
[20 tokens; log P(seq): -282.65]

<s> <s> during the war drum steps other cared unaware decorating hollering magazines pious merciless integrals tropic sp-44006 disparate day-watch mansions rhythms
[20 tokens; log P(seq): -260.05]

<s> <s> armed curbside liquid tragic admissible naebm people drains magnetic limited twentieth-century column belly fronti

## Scoring on Held-Out Data

Try playing with the "k" parameter above to see how it impacts the result. 

In [43]:
log_p_data, num_real_tokens = lm.score_seq(train_tokens)
print("Train perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

log_p_data, num_real_tokens = lm.score_seq(test_tokens)
print("Test perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

Train perplexity: 43.05
Test perplexity: 3599.52


## Part (e): Additional Questions

Answer the following in the cell below.

1. What is the average number of times our model sees any particular trigram in the training set, averaged across the trigrams we observed at least once (i.e. ignoring the zeros for trigrams we never observed)? How about averaged across *all possible* trigrams (i.e. including hypothetical ones we never observed)? (*Hint:* you don't need to write any code for this - it should be a quick calculation.) (*Second Hint:* you can factor in the start and end sentence tags or ignore them, we'll accept either answer.)
<p>
2. Based on your answer above, do you think that a 5-gram model would perform better than the trigram model on the 1 million token Brown corpus?  
<p><p>
3. Which model generates more "realistic" sentences - `AddKTrigramLM` or the unsmoothed `SimpleTrigramLM` from the demo notebook?

## Just for fun: Linguistic Curiosities

You might have seen this floating around the internet:
![Adjective Order](adjective_order.jpg)
*source: https://twitter.com/MattAndersonBBC/status/772002757222002688?lang=en*

Let's see if it holds true, statistically at least. Note that log probabilities are always negative, so the smaller magnitude is better. And remember the log scale: a difference of score of 8 units means one utterance is $2^8 = 256$ times more likely!

For this part, we'll use a trigram model (but with a more sophisticated smoothing technique, **Knesser-Ney**, which we learned about in the async).

In [30]:
import kn_lm
reload(kn_lm)

Model = kn_lm.KNTrigramLM
t0 = time.time()
print("Building KN trigram LM... ", end="")
lm = Model(train_tokens)
print("done in {:.02f} s".format(time.time() - t0))
lm.print_stats()

Building KN trigram LM... done in 4.89 s
=== N-gram Language Model stats ===
  30,000 unique 1-grams
 357,896 unique 2-grams
 732,467 unique 3-grams
Optimal memory usage (counts only): 24.21 MB


In [31]:
def preprocess_for_scoring(sentence, vocab):
    # Pre-process words, replace anything the model doesn't know with <unk>
    words = [utils.canonicalize_word(w, wordset=vocab.wordset)
             for w in sentence]
    # Pad sequence with start and end markers
    return [u"<s>", u"<s>"] + words + [u"</s>"]

s0 = preprocess_for_scoring("square green plastic toys".split(), vocab)
s1 = preprocess_for_scoring("plastic green square toys".split(), vocab)

In [32]:
print("s0 score: {:.02f}".format(lm.score_seq(s0)[0]))
print("s1 score: {:.02f}".format(lm.score_seq(s1)[0]))

s0 score: -52.01
s1 score: -60.96


In [33]:
noun = "toys"
adjectives = ["square", "green", "plastic"]
results = []
for adjs in itertools.permutations(adjectives):
    words = list(adjs) + [noun]
    seq = preprocess_for_scoring(words, vocab)
    score, _ = lm.score_seq(seq)
    results.append((score, words))

# Sort results
for score, words in sorted(results, reverse=True):
    print("\"{:s}\" : {:.02f}".format(" ".join(words), score))

"green square plastic toys" : -51.95
"square green plastic toys" : -52.01
"plastic square green toys" : -60.96
"plastic green square toys" : -60.96
"green plastic square toys" : -61.25
"square plastic green toys" : -61.31
